# The first part of the exercise is a high level application of the logistic regression using the "sklearn" package
# In the second part I try to implement my own logistic regression algorithm and show that it is better at predicting the versicolor class vs all others 
# I am using gradient descent because there is no closed form solution for logistic regression, but I highly optimize it using the "scipy.optimize" package

In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize

In [2]:
mat = loadmat('iris_multiclass.mat')
meas = mat['meas'] # measurements: length sepals, width sepals, lenght petals, width petals
species = mat['species']

In [3]:
setosa_x_train = meas[0:40]
setosa_x_test = meas[40:50]

versicolor_x_train = meas[50:90]
versicolor_x_test = meas[90:100]

virginica_x_train = meas[100:140]
virginica_x_test = meas[140:150]

In [4]:
vv_train = np.concatenate((versicolor_x_train, virginica_x_train), axis=0)
X_train = np.concatenate((setosa_x_train, vv_train), axis=0)

vv_test = np.concatenate((versicolor_x_test, virginica_x_test), axis=0)
X_test = np.concatenate((setosa_x_test, vv_test), axis=0)

In [5]:
setosa_y_train_1d = np.append(np.ones((40, 1), dtype=int), np.zeros((80, 1), dtype=int))
setosa_y_test_1d = np.append(np.ones((10, 1), dtype=int), np.zeros((20, 1), dtype=int))

versicolor_y_train_1d = np.append(np.zeros((40, 1), dtype=int), [np.ones((40, 1), dtype=int), np.zeros((40, 1), dtype=int)])
versicolor_y_test_1d = np.append(np.zeros((10, 1), dtype=int), [np.ones((10, 1), dtype=int), np.zeros((10, 1), dtype=int)])

virginica_y_train_1d = np.append(np.zeros((80, 1), dtype=int), np.ones((40, 1), dtype=int))
virginica_y_test_1d = np.append(np.zeros((20, 1), dtype=int), np.ones((10, 1), dtype=int))

In [6]:
setosa_lr = LogisticRegression()
setosa_lr.fit(X_train, setosa_y_train_1d)
print('Score Setosa: ', setosa_lr.score(X_test, setosa_y_test_1d))

versicolor_lr = LogisticRegression()
versicolor_lr.fit(X_train, versicolor_y_train_1d)
print('Score Versicolor: ', versicolor_lr.score(X_test, versicolor_y_test_1d))

virginica_lr = LogisticRegression()
virginica_lr.fit(X_train, virginica_y_train_1d)
print('Score Virginica: ', virginica_lr.score(X_test, virginica_y_test_1d))

Score Setosa:  1.0
Score Versicolor:  0.633333333333
Score Virginica:  1.0


# Low Level Details (from scratch...)

In [7]:
setosa_y_train = np.append(np.ones((40, 1), dtype=int), np.zeros((80, 1), dtype=int), axis=0)
setosa_y_test = np.append(np.ones((10, 1), dtype=int), np.zeros((20, 1), dtype=int), axis=0)

intermediate_train = np.append(np.ones((40, 1), dtype=int), np.zeros((40, 1), dtype=int), axis=0)
intermediate_test = np.append(np.ones((10, 1), dtype=int), np.zeros((10, 1), dtype=int), axis=0)
versicolor_y_train = np.append(np.zeros((40, 1), dtype=int), intermediate_train, axis=0)
versicolor_y_test = np.append(np.zeros((10, 1), dtype=int), intermediate_test, axis=0)

virginica_y_train = np.append(np.zeros((80, 1), dtype=int), np.ones((40, 1), dtype=int), axis=0)
virginica_y_test = np.append(np.zeros((20, 1), dtype=int), np.ones((10, 1), dtype=int), axis=0)

#  Hypothesis of the logistic regression:
$$h_{\theta}(x) = g(\theta^{T}x)$$
# The "Sigmoid" function from neural nets, also called "Logistic" Function, because it is useful for calculating the logistic regression probability: 
$$g(z)=\frac{1}{1+e^{−z}}$$

In [8]:
def sigmoid(z):
    return 1.0/(1+np.exp(-z))

# The sigmoid function above basically calculates the probability of y=1 given input X and parameters theta of the model

# Cost function in vector form: $$J(\theta) = \frac{1}{m}\big((\,log\,(h)^Ty+(\,log\,(1-h)^T(1-y)\big)$$

In [9]:
def costFunction(theta, X, y):
    m = y.size
    h = sigmoid(X.dot(theta))
    J = (-1/m)*(np.log(h).T.dot(y) + np.log(np.subtract(np.ones_like(h), h)).T.dot(np.subtract(np.ones_like(y), y)))
    if np.isnan(J[0]):
        return np.inf
    return J[0]

# Partial Derivative of the cost function:$$\frac{\delta J(\theta)}{\delta\theta_{j}} = \frac{1}{m} X^T(h-y)$$

In [11]:
def gradient(theta, X, y):
    m = y.size
    h = sigmoid(X.dot(theta.reshape(-1,1)))
    grad = (1/m) * X.T.dot(np.subtract(h, y))
    return grad.flatten()

In [15]:
def predict(theta, X, threshold=0.5):
    p = sigmoid(X.dot(theta.T)) >= threshold
    return(p.astype('int'))

In [12]:
initial_theta = np.zeros(X_train.shape[1])
cost = costFunction(initial_theta, X_train, setosa_y_train)
grad = gradient(initial_theta, X_train, setosa_y_train)

In [13]:
print('Cost: \n', cost)
print('Grad: \n', grad)

Cost: 
 0.69314718056
Grad: 
 [ 1.26583333  0.38125     1.41083333  0.5175    ]


In [14]:
res_setosa = minimize(costFunction, initial_theta, args=(X_train, setosa_y_train), method=None, jac=gradient, options={'maxiter':400})
print(res_setosa)

      fun: 1.2855236822525272e-06
 hess_inv: array([[  16551.75808348,   50788.88380548,  -73200.43422433,
         -32901.62518952],
       [  50788.88380548,  155852.12532391, -224621.48333733,
        -100961.42705816],
       [ -73200.43422433, -224621.48333733,  323737.18900477,
         145510.9347199 ],
       [ -32901.62518952, -100961.42705816,  145510.9347199 ,
          65404.34872674]])
      jac: array([  7.13053707e-06,   3.41378695e-06,   4.51086492e-06,
         1.40681969e-06])
  message: 'Optimization terminated successfully.'
     nfev: 23
      nit: 21
     njev: 23
   status: 0
  success: True
        x: array([  2.56870677,   7.73804093, -11.18151263,  -5.0383167 ])


In [16]:
p = predict(res_setosa.x, X_test)
print('Test accuracy for Setosa {}%'.format(100*sum(p == setosa_y_test.ravel())/p.size))

Test accuracy for Setosa 100.0%


In [17]:
res_versicolor = minimize(costFunction, initial_theta, args=(X_train, versicolor_y_train), method=None, jac=gradient, options={'maxiter':400})
print(res_versicolor)
p = predict(res_versicolor.x, X_test)
print('Test accuracy for Versicolor {}%'.format(100*sum(p == versicolor_y_test.ravel())/p.size))

      fun: 0.5354954335365321
 hess_inv: array([[  40.20233584,  -46.27041686,  -36.54017523,   35.23056497],
       [ -46.27041686,   58.04239404,   35.10232899,  -28.24348681],
       [ -36.54017523,   35.10232899,   53.43628325,  -80.03652787],
       [  35.23056497,  -28.24348681,  -80.03652787,  154.74735142]])
      jac: array([ -8.58119592e-06,  -2.85362203e-06,  -6.55055339e-06,
        -3.83252281e-06])
  message: 'Optimization terminated successfully.'
     nfev: 25
      nit: 22
     njev: 25
   status: 0
  success: True
        x: array([ 1.07961973, -2.12630182,  0.24571297, -1.40560459])
Test accuracy for Versicolor 66.66666666666667%


In [18]:
res_virginica = minimize(costFunction, initial_theta, args=(X_train, virginica_y_train), method=None, jac=gradient, options={'maxiter':400})
print(res_virginica)
p = predict(res_virginica.x, X_test)
print('Test accuracy for Virginica {}%'.format(100*sum(p == virginica_y_test.ravel())/p.size))

      fun: 0.08800879052227342
 hess_inv: array([[  704.75558395,   310.49198632,  -948.59108984,  -352.82117833],
       [  310.49198632,   761.51996752,  -590.14521628,  -732.65069561],
       [ -948.59108984,  -590.14521628,  1420.71133303,   352.23065183],
       [ -352.82117833,  -732.65069561,   352.23065183,  1541.39951246]])
      jac: array([  8.50351490e-06,   4.83610612e-06,   6.87213793e-06,
         2.95576766e-06])
  message: 'Optimization terminated successfully.'
     nfev: 43
      nit: 41
     njev: 43
   status: 0
  success: True
        x: array([-6.16375466, -6.14473363,  8.34317457,  9.09723037])
Test accuracy for Virginica 100.0%


# Note the 66% accuracy vs 63% accuracy in favor of the optimized gradient descent algorithm